# Makine Öğrenmesi İle Tablet Sınıflandırma

İhtiyacımız olan kütüphaneleri çekirdeğe ekliyoruz. 

In [ ]:
#Keşifçi Veri Analizi
import numpy as np
import pandas as pd
#Görselleştirme
import seaborn as sns
import matplotlib.pyplot as plt
#Veri Ön İşleme
import missingno                  
from sklearn import preprocessing
#Makine Öğrenmesi Naive Bayes
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.naive_bayes import GaussianNB
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
#KNN
from sklearn.neighbors import KNeighborsClassifier

Veri çerçevemizi pd.read ile okuyoruz. Ve istediğimiz zaman geri dönmek için kopyalıyoruz.

In [ ]:
tablet=pd.read_csv("../input/tabletpc-priceclassification/tablet.csv")
df=tablet.copy()

# KEŞİFÇİ VERİ ANALİZİ

İlk 5 gözlem.

In [ ]:
df.head()

Veri çerçevemizin bilgelirini yazdırıyoruz. RAM ve ön kamerada eksikler olduğunu görüyoruz.

In [ ]:
df.info()

2000 gözlem 20 öznitelik olduğunu görüyoruz.

In [ ]:
df.shape

Eksik verilerimiz var mı diye kontrol ediyoruz. Göreceğimiz üzere 5 tane ön kamera 12 tane ram kolonunda eksiklerimiz var.

In [ ]:
df.isna().sum()

Kaç tane veri var her birinden?

In [ ]:
df.count()

Fiyat aralıklarına bakalım

In [ ]:
df["FiyatAraligi"]

Fiyat aralığının unique değerleri

In [ ]:
df["FiyatAraligi"].unique()

4G'nin unique değerleri.

In [ ]:
df["4G"].unique()

Çekirdek sayısı unique değerleri.

In [ ]:
df["CekirdekSayisi"].unique()

Ramin unique değerleri.

In [ ]:
df["RAM"].nunique()#bunlardan bir tanesi nan çünkü 12 tane eksik veri olduğunu biliyoruz.

In [ ]:
df["ArkaKameraMP"].unique()

Renk çeşitleri.

In [ ]:
df["Renk"].unique()#renkler

Batarya ömürleri.

In [ ]:
df["BataryaOmru"].unique()

Kaç renk çeşidi olduğuna bakalım.


In [ ]:
df["Renk"].nunique()

1019 tane çift hat olan 981 cift hat olmayan tablet var.

In [ ]:
df["CiftHat"].value_counts()

1010 tane bluetooth olan 990 tane bluetooth olmayan tablet var. 

In [ ]:
df["Bluetooth"].value_counts()

Hedef değişken dengeli dağılmış mı?

In [ ]:
df["FiyatAraligi"].value_counts()#Kesinlikle dengelidir.

İstatiksel çıkarımlar

In [ ]:
df.describe().T
#Batarya gücünün standart sapması minimum değerinden küçüktür. Medyan değeri ile aritmetik ortalama değeri çok yakındır. Verilerin batarya gücü değerleri birbirine yakındır.
#Kalınlığın standart sapması en düşüktür ortalamaya en yakın olan değerler o özniteliktedir. 
#Çözünürlük genişliğin standart sapması minumum değerinden küçüktür. Yine ortalama değeri medyana çok yakındır. Verilerin CozunurlukGenislik değerleri birbirine yakındır.
#Ağırlığın standart sapması minumum değerinden küçüktür. Medyan değeri ile ortalama değeri yakındır. Tabletlerin ağırlığı birbirine yakındır.

Veri setimizin en çok tekrar eden değerleri.

In [ ]:
df.mode()

Fiyat aralıklarına göre öznitelik ortalamaları.

In [ ]:
df.groupby(df["FiyatAraligi"]).mean()#ucuz tabletlerin batarya gücü ve ömrü; çözünürlük yükseklik ve çözünürlük genişlik; dahili bellek ortalaması  normal tabletlerden iyi, geri kalanı normal fiyatlı tabletlerden iyi.  

Fiyat aralığına göre batarya gücünün istetiksel değerleri.

In [ ]:
df.groupby(df["FiyatAraligi"]).describe()["BataryaGucu"]

Batarya gücü 1350'den büyük, çekirdek sayısı 7'den buyuk, fiyat aralığı ucuz olan değerlerin batarya ömürlerine göre azalan sırada sıralanması.

In [ ]:
df[(df['CekirdekSayisi']>7) & (df['FiyatAraligi'] == "Ucuz")&(df["BataryaGucu"]>1350)].sort_values('BataryaOmru', axis = 0, ascending=False)

Bluetooth'un, WiFi'nin, 3G'nin, 4G'nin var oldukları ve yok oldukları durumlarda fiyat aralıklarına göre RAM'in, mikro işlemci hızının ve çekirdek sayısının ortalamaları.   

In [ ]:
pd.pivot_table(df, values = ["RAM", "CekirdekSayisi","MikroislemciHizi"], index = ["Bluetooth", "WiFi", "3G", "4G"],columns = ["FiyatAraligi"], aggfunc = np.mean)

Özniteliklerin ilişkilerinin yönü için kovaryans tablosu.

In [ ]:
df.cov()

Korelasyon tablosu. -1 ile 1 arasında değerler, ilişkiler güçlü negatif yönlü ilişkiden güçlü pozitif ilişkiye kadar değişkenlik gösterir.

In [ ]:
df.corr()
#ArkaKameraMP, OnKameraMP aralarındaki korelasyon 0,65'dir, orta seviye pozitif yönlü bir ilişkidir.
#CozunurlukYukseklik ve CozunurlukGenislik arasında 0,51'lik orta seviye pozitif yönlü bir ilişki vardır.

# Görselleştirme

In [ ]:
sns.set(rc={'figure.figsize':(11,6)})  # grafikleri 11, 6 şeklinde boyutlandırıyoruz.

In [ ]:
sns.scatterplot(x = "ArkaKameraMP", y = "OnKameraMP" ,data = df);#arka kamera ve ön kameranın grafiği pozitif yönde artış var.

Korelasyon matrisi ısı haritası şeklinde.

In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values);#ArkaKameraMP ve OnKameraMP arasında pozitif yönlü orta seviye bir ilişki vardır.
                                             #CozunurlukYukseklik ve CozunurlukGenislik arasında pozitif yönlü düşük seviye bir ilişki vardır

Fiyatlarına göre mikro işlemci hızı ve RAM ilişkisi.

In [ ]:
sns.scatterplot(x = "RAM", y = "MikroislemciHizi" ,hue="FiyatAraligi",data = df);#RAM arttıkça fiyatın da arttığını söyleyebiliriz.

Verileri histogram üzerinde

In [ ]:
df.hist(figsize = (20,20))

plt.show()

RAM boyutlarının verisetindeki dağılımı.

In [ ]:
sns.distplot(df["RAM"], bins=16, color="Brown");#Birbilerine yakın frekanstalar ancak 1250 ile 1500 arasında olan RAM boyutları grafikte bulunan diğer aralıklardan büyüktür.

Dahili bellek boyularının veri setindeki dağılımı.

In [ ]:
sns.distplot(df["DahiliBellek"], bins=16, color="Brown");#Dahili bellekte 32-36 gb arası dahili bellek boyutundan verisetimizde diğerler aralıklardan daha az vardır. 

In [ ]:
sns.jointplot(x = df["RAM"], y = df["DahiliBellek"], kind = "kde", color = "brown");#Dahili bellek 10 ile 20 arasındayken ve RAM boyutu 2000 ile 2500 arasındayken yoğunluk fazladır.

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "BataryaGucu",  data = df);#Batarya güçleri; 
                                                                  #Pahalı tabletlerde 1750 ve 2000 arası yoğunlaşır.
                                                                  #Çok ucuz  tabletlerde 500 ile 1000  arası yoğunlaşır.
                                                                  #Ucuz  tabletlerde 1000 ile 1500 arası yoğunlaşır.
                                                                  #Normal tabletler dengelidir.

In [ ]:
sns.barplot(x ="Renk" , y = "OnKameraMP", data = df);#Sarı rengin ön kamerası daha iyi. Pembe rengin ön kamerası diğerlerinden daha kötüdür. 

In [ ]:
 sns.jointplot("ArkaKameraMP", "OnKameraMP", data = df,color="red");#Ön kameranın olmadığı ve arka kameranın olduğu değerler yoğunluktadır. 

# Veri Ön İşleme

Tekrar on kamerada 5 ve RAM'de 12 taneeksik verimizin olduğunu hatırlayalım. 

In [ ]:
df.isna().sum()

Eksik değerlerinin hangi öznitelikte ve hangi aralıkta oluğunu gösterelim.

In [ ]:
missingno.matrix(df,figsize=(10, 8));

Eksik değerlerin arsındaki ilişki ne kadar ve ne yönde.

In [ ]:
missingno.heatmap(df, figsize= (8,8));#İlişki yoktur eksik veriler rastgeledir.

Eksik verilerin benzersiz değerlerini gösterelim.

In [ ]:
df["RAM"].unique()#çok fazla değer olduğu için nan değerini göremeyiz ama vardır.

In [ ]:
df["OnKameraMP"].unique()#nan değerinin olduğunu görürüz.

Eksik verilerin neden girilmediğine bakalım. Bunun için diğer öznitelik değerlerini de görelim.

Ön kamerası olmadığı için mi nan girilmiştir?  

In [ ]:
df[df["OnKameraMP"].isnull()]#Ortak noktası çok ucuz olmalarıdır. Bundan dolayı belki de ön kamerası yoktur. 3G olması da ortaktır ama ön kamerayla 3G'nin ilişkisi yoktur.  
#bir tane eksik verinin ArkaKamerası da yoktur.

Eksik verilerin indislerine bulalım.

In [ ]:
eksik_OnKameraVerileriIndexleri = df[df["OnKameraMP"].isnull()].index

In [ ]:
eksik_OnKameraVerileriIndexleri

In [ ]:
sns.countplot(df[df["OnKameraMP"].isnull()]["Bluetooth"]); #Bluetooth 4 tabletin yoktur. Bu ucuzluk için bir göstergedir.

Eksik veriler bazı özniteliklerin ortalamalarının altında mı? (Ucuzluk belirtisi.) 

In [ ]:
df[df["OnKameraMP"].isnull()]["RAM"]

In [ ]:
df["RAM"].mean()#Hepsinin Ram'i bu değerlerin altındadır. 

In [ ]:
df[df["OnKameraMP"].isnull()]["BataryaOmru"]

In [ ]:
df["BataryaOmru"].mean()#biri hariç hepsinin batarya ömürlerinin az olduğunu görürüz. O tabletin ne WiFisi ne dokunmatiği ne ArkaKamerası ne de 4G'si vardır.

Bu eksik verilerin çok ucuz oldukları için değerlerinin girilmediğine ulaşırız. Olasığı en yüksek olan değerler ile yerlerini değiştireceğiz.

In [ ]:
df.loc[eksik_OnKameraVerileriIndexleri,"OnKameraMP"] = 0 #Çok ucuz oldukları için ön kameraları yoktur. O yüzden 0 gireriz.

In [ ]:
df["OnKameraMP"].isna().sum() #Eksik veri bulunuyor mu? 

In [ ]:
df["OnKameraMP"].unique()#nan değerinin olmadığını görürüz.

RAM'in eksik değer tablosu gösterelim

In [ ]:
df[df["RAM"].isnull()]

İndexlerini değişkene atıyoruz.

In [ ]:
eksik_RAMVerilerininIndexleri = df[df["RAM"].isnull()].index

In [ ]:
eksik_RAMVerilerininIndexleri#RAM'in eksik verilerin indexleri

Bu veriler neden eksik girilmiştir? Yanlışlıkla eksik girilmiş olabilir mi?

In [ ]:
sns.countplot(df[df["RAM"].isnull()]["MikroislemciHizi"]);#Mikro işlemci hızı 1.8 olan 3 tane 0.5 olan 2 tane değer vardır. Karışık olduğunu görürüz.

In [ ]:
sns.countplot(df[df["RAM"].isnull()]["CekirdekSayisi"]);#Cekirdek sayisi 8 olan 2 tane tablet var, 1 olan da 2 tablet vardır. Bu da bizi bir sebebe ulaştırmıyor. 

In [ ]:
sns.countplot(df[df["RAM"].isnull()]["Dokunmatik"])#Bu grafikte dokunmatiği olan ve dokunmatiği olamayan tabletlerin sayısı eşittir. Buradan da eksik girilmesinin sebebine ulaşamayız.

In [ ]:
df[df["RAM"].isnull()]["BataryaGucu"] 

In [ ]:
df["BataryaGucu"].mean()#7 tanesi ortalamanın üstünde 5 tanesi altındadır. Yine eksik verilerin sebebini bulamadık. 

In [ ]:
df[df["RAM"].isnull()]["DahiliBellek"]

In [ ]:
df["DahiliBellek"].mean()#Yine ortalamanın üstünde ve altında dahilli bellek değerleri bulunur.

Eksik değerlerin bir özniteliğe göre nan olamasından çok yanlışlıkla girilmiştir. Yani bir sebebi yoktur. Bu yüzden ortalama değer ile yerlerini dolduracağız. 

Doldurma işlemini pahalı fiyat aralığındaki ortalamaya göre yapacağız. Çünkü bütün eksik veriler pahalıdır. Böylece girilmek istenen değerlere daha çok yaklaşacağız.  

Pahalı fiyat aralığında olduğu için bütün veriler ve makine öğrenmesi modeli daha sağlıklı tahminler yapması için değerler bu aralıkta kalmalıdır. O yüzden genel RAM ortalaması yerine pahlı aralıktaki RAM ortalaması ile  doldurulur. Bu öznitelik makine öğrenmesi modelinde bağımsız değişkenlerden biridir.

In [ ]:
df[df["FiyatAraligi"] == "Pahalı"]["RAM"].mean()#verisetindeki fiyat aralığı pahalı olan verilerin RAM ortalamasıdır.

In [ ]:
df.loc[eksik_RAMVerilerininIndexleri,"RAM"] = 3450 #Eksik verilerimizi olma olasılğı yüksek olan verilerle değiştirdik.

Hâlâ boş değer var mı?

In [ ]:
 df["RAM"].isna().sum()

In [ ]:
 df.isna().sum().sum()

Kategorik değişkenlerin ordinal olanlarını label Encoder ile nominal olanları get dummies ile sayısallaştıracağız.

In [ ]:
df['Renk'] = pd.Categorical(df['Renk'])
dfDummies = pd.get_dummies(df['Renk'], prefix = 'Renk')#prefix dememizin sebebi kolonun özniteliğin Renk ile başlamasını istememiz
dfDummies
#get dummies'i kullanıyoruz çünkü renk nominaldir, bir rengin diğerine üstünlüğü yoktur.

In [ ]:
df = pd.concat([df, dfDummies], axis=1)#Bu verileri verisetimiz ile birleştiriyoruz.
df.head()

Şimdi Renk kolonunu silelim.

In [ ]:
df.drop(["Renk"], axis=1, inplace = True)


In [ ]:
label_encoder = preprocessing.LabelEncoder() #label encoder'ımızı tanımlıyoruz. Label encoderı kullandığımız yerlerde birince veriyi baz alır.

Label encoderı ordinal değerlerde yani bir verinin diğerine üstünlük kurabileceği durumlarda kullanırız.

Bu değiştireceğimiz özniteliklerin ordinal olmasınının sebebi bir tableti almamızda önemli öznitelikler olması.

In [ ]:
df['FiyatAraligi'] = label_encoder.fit_transform(df['FiyatAraligi'])#bizim ilk fiyat aralığı değerimiz normal olduğu için 0 normaldir.1 pahalı, 2 ucuz, 3 çok ucuzdur. 
df["FiyatAraligi"]

In [ ]:
df['Bluetooth'] = label_encoder.fit_transform(df['Bluetooth']) 
df["Bluetooth"]#Yok 1'dir.Var 0'dır

In [ ]:
df['CiftHat'] = label_encoder.fit_transform(df['CiftHat']) 
df["CiftHat"]#Yok 1'dir.Var 0'dır.

In [ ]:
df['4G'] = label_encoder.fit_transform(df['4G']) 
df["4G"]#Yok 1'dir.Var 0'dır.

In [ ]:
df['3G'] = label_encoder.fit_transform(df['3G']) 
df["3G"]#Yok 0'dır.Var 1'dir.

In [ ]:
df['Dokunmatik'] = label_encoder.fit_transform(df['Dokunmatik']) 
df["Dokunmatik"]#Yok 0'dır.Var 1'dir.

In [ ]:
df['WiFi'] = label_encoder.fit_transform(df['WiFi']) 
df["WiFi"]#Yok 0'dır.Var 1'dir.

# Makine Öğrenmesi

Bağımlı(y) ve bağımsız(X) değişkenleri bölütleyelim.

In [ ]:
y = df['FiyatAraligi']#fiyat aralığı bağımlı diğerleri bağımsız değişkendir
X = df.drop(['FiyatAraligi'], axis=1)

Bağımsız ve bağımlı değişkenlerimiz

In [ ]:
y

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size = 0.25, 
                                                    random_state =42)#%25 test verisi olacak %75 eğitim verisi olacaktır. 
#Random State ise her defasında karıştırmayı engeller.Hep aldığımız sonucu alacağız.

Test Verilerimiz.

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

Modeli yaratıyoruz.

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)

X değişkeninin rastgele 10 değerini modelimizin tahmin etmesi için parametre olarak verelim.

In [ ]:
nb_model.predict(X.sample(10))

Modele bağımlı test değişkenini tahmin ettirelim. Bu tahmin değerlerimizi y_pred değişkenine atayalım.

In [ ]:
y_pred = nb_model.predict(X_test)

In [ ]:
y_pred

Karmaşıklık matrisini görüntüleyelim. 

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred) 
print(karmasiklik_matrisi)

labelencoder ile atadığımız değerler;                                                                                                                                  
0--> normal 1--> pahalı 2--> ucuz  3-->çok ucuz
                                                                                                                                                                                                                               
Test verilerimizin;

Fiyat aralığı normal olanlardan;                                                                                                                                                                                         
80 tanesi doğru tahmin etmiştir,                                                                                                                                                                                                      
27 tanesini pahalı tahmin etmiştir,                                                                                                                                                             
25 tanesini ucuz tahmin etmiştir,                                                                                                                                                              
Hiçbirini çok ucuz tahmin etmemiştir.                                                                                                                                                

Fiyat aralığı pahalı olanlardan;                                                                                                                               
85 tanesi doğru tahmin etmiştir,                                                                                                                                                          
19 tanesini normal tahmin etmiştir,                                                                                                                                                                
Hiçbirini ucuz tahmin etmemiştir,                                                                                                                                     
Hiçbirini çok ucuz tahmin etmemiştir.
                                                                                                                                                                             
Fiyat aralığı ucuz olanlardan;                                                                                                                                                                    
90 tanesi doğru tahmin etmiştir,                                                                                                                                                   
32 tanesini normal tahmin etmiştir,                                                                                                                                                             
1 tanesini pahalı tahmin etmiştir,                                                                                                                                                           
11 tanesini çok ucuz tahmin etmiştir.                                                                                                                                                          


Fiyat aralığı çok ucuz olanlardan;                                                                                                                                                                                              
118 tanesi doğru tahmin etmiştir,                                                                                                                                                     
Hiçbirini normal tahmin etmemiştir,                                                                                                                                                                
Hiçbirini pahalı tahmin etmemiştir,                                                                                                                                                                   
12 tanesini ucuz tahmin etmiştir.                                                                                                                                                         

Gerçek değer ile ne kadar yakın olduğuna bakalım.

In [ ]:
accuracy_score(y_test, y_pred)#Verimiz dengeli olduğu için accuracy komutunu kulanırız. Accuracy doğru tahmin edilen verilerin tüm tahmin edilen verilere oranını verir.

In [ ]:
print(classification_report(y_test, y_pred))

Matrisimiz 4x4 olduğu için doğru pozitif tahmin edilen de gerçek de kendisidir. Yanlış pozitif tahmin edilen kendisi ama gerçek değeri kendisi değildir. Yanlış negatif ise gerçek değeri kendisi ama tahmin edilen değeri farklıdır.                                                                                                                                                                                                                   Örneğin Normal için precision=80/(80+27+25+0)=0.61                                                                                                                                                                           

Normal fiyat aralığında tahmin ettiğimiz verilerin %61'i doğrudur. --> Precision                                                                                                                                                                                                                                                                                                                                                                                                                                                         
Gerçekte normal fiyat aralığında olan değerlerin %61'i doğru tahmin edilmiştir.-->Recall                                                                                                                                                                       
F1 skoru bu iki değerin ağırlıklı ortalamasıdır. Model bu aralıkta %61 başarılıdır.

Pahalı fiyat aralığında tahmin ettiğimiz verilerin %82'si doğrudur.                                                                                                                                                                                                                                                                                                                                                                                                                                                          
Gerçekte pahalı fiyat aralığında olan değerlerin %75'i doğru tahmin edilmiştir.                                                                                                                                                                       
Model bu aralıkta %78 başarılıdır.

Ucuz fiyat aralığında tahmin ettiğimiz verilerin %67'si doğrudur.                                                                                                                                                                                                                                                                                                                                                                                                                                                          
Gerçekte ucuz fiyat aralığında olan değerlerin %71'i doğru tahmin edilmiştir.                                                                                                                                                                       
Model bu aralıkta %69 başarılıdır.

Çok ucuz fiyat aralığında tahmin ettiğimiz verilerin %91'i doğrudur.                                                                                                                                                                                                                                                                                                                                                                                                                                                          
Gerçekte çok ucuz fiyat aralığında olan değerlerin %91'i doğru tahmin edilmiştir.                                                                                                                                                                       
Model bu aralıkta %91 başarılıdır.

Sonuç olarak modelimiz uç noktalarda bariz daha başarılıdır. Çünkü uç noktalardaki veriler belirgindir. Pahalı ve çok ucuz bu yüzden diğerlerinden yüksek çıkmıştır.     
                                                                                                                                                                                                                                        
Bu arada biz eksik ön kamera verilerini 0 girmiştik, bu uç noktadaki verilerimizi daha belirgin yapar.                        
                                                                                                                                                                                                                               
Pahalı fiyat aralığında olan eksik ram değerlerini de ortalama girdik bu da pahalı aralığındaki verilerin belirginliğini azaltır.           

Decision Tree modelimizi kriteri gini(default) alarak yaratıyoruz.

In [ ]:
cart = DecisionTreeClassifier(random_state = 42) #Varsayılan sınıflandırma kriteri gini'dir.
cart_model = cart.fit(X_train, y_train)

Modele bağımsız test değişkenini yolluyoruz.

In [ ]:
y_pred = cart_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

Decision tree (gini) başarı skoru.

In [ ]:
accuracy_score(y_test, y_pred)

Decision tree (gini) karmaşıklık matrisimiz.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Decision Tree modelimizi kriteri entropi alarak yaratıyoruz. 

In [ ]:
cart = DecisionTreeClassifier(random_state = 42, criterion='entropy')
cart_model = cart.fit(X_train, y_train)

Modele bağımsız test değişkenini yolluyoruz.

In [ ]:
y_pred = cart_model.predict(X_test)

Decision tree (entropi) karmaşıklık matrisimiz.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Decision tree (entropi) başarı skoru.

In [ ]:
accuracy_score(y_test, y_pred)#Criterion gini iken başarı skoru %81 'di. Entropi ile değiştirince %87 oldu. Entropi parametresi ile modelimiz daha başarılıdır.

In [ ]:
print(classification_report(y_test, y_pred))#Görüleceği üzere Entropi parametresi ile modelimiz daha başarılıdır.

KNN modelimiz yaratıyoruz.

In [ ]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)

Knn modelinin alabileceği parametreler, burada n_neighbours parametresini kullanacağız. En yakın komşu sayısını parametre olarak girmezsek 5 olarak alır.

In [ ]:
knn_model

Modele bağımsız test değişkenini yolluyoruz.

In [ ]:
y_pred = knn_model.predict(X_test)

Knn karmaşıklık matrisi.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Başarı skoru.

In [ ]:
accuracy_score(y_test, y_pred)#Knn varsayılan olarak 5 aldığımızda başarı skoru %93'tür. Şu ana kadarki modellerin başa skorlarından daha doğrudur.

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
knn_accuracy_score=list()#Başarı skorlarımızı bir listede tutacağız. 
for i in range(2,15):
    knn = KNeighborsClassifier(n_neighbors=i)#modelin en yakın i komşusunu parametre olarak alacaktır.i 2'den 15'e kadar gider.
    knn_model = knn.fit(X_train, y_train)
    y_pred = knn_model.predict(X_test)
    knn_accuracy_score.insert(i, accuracy_score(y_test, y_pred))#insert metoduyla listemize başarı skorlarını ekliyoruz.

In [ ]:
   knn_accuracy_score 

Bu başarı skorunu grafikle görüntüleyelim.

In [ ]:
plt.plot(knn_accuracy_score[0:15])
plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11,12])#En yüksek başarı skorunu değerini 9 indexli eleman ile ulaşılmıştır.
plt.show()

En yüksek başarı skoru.

In [ ]:
knn_accuracy_score[9]# En yüksek başarı skoru 0.938'dir.

Varsayılan en yakın komşu sayısı parametresi 5'tir. Ancak biz 2 den itibaren kaydetmmeye başladığımız için knn_accuracy_score listemizde 3.indextir.

In [ ]:
knn_accuracy_score[3]

Optimum komşu sayısı ile modeli bir daha eğitelim. 

In [ ]:
knn = KNeighborsClassifier(11)#9 yerine 11 vermemizin sebebi listedeki indexleri 2 den başlatmış olmamızdır.
knn_model = knn.fit(X_train, y_train)

In [ ]:
y_pred = knn_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

Varsayılan en yakın komşu ile ulaştığımız başarı skoru 0.93'tür. 11 en yakın komşu ile ulaştığımız başarı skoru 0.938'dir. KNN modeli her en yakın komşu sayısı için aynı sonucu vermez, Belli bir komşu sayısına kadar artar (bizim modelimzde 11) sonra azalır.

In [ ]:
print(classification_report(y_test, y_pred))

f1 skorlarını karşılaştırdığımızda iki knn modelinin normal ve ucuz tabletlerde aynı oranda doğru tahmin etseler de 11 en yakın komşulu knn modeli pahalı ve çok ucuz tabletlerde daha yüksek oranda doğru tahmin etmiştir. 

Şu ana kadar ki en doğru sonucu veren model 11 en yakın komşulu KNN modelidir.